In [1]:
# importo le librerie
import pandas as pd 
import ast
import json
import numpy as np
# importo il dataset
movies = pd.read_csv('tmdb_5000_movies.csv')
df_movies = movies.rename(columns = {'id' : 'movie_id'})
df_movies.head(2)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [2]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head(6)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."


In [3]:
#funzione estrazione colonne json
def estrazione_json(db, column):
    result = db[column].apply(pd.read_json)
    movie_id = db['movie_id'] 
    for i in range(len(db[column])):
        result[i]['movie_id'] = movie_id[i]
    return result.pipe(lambda x: pd.concat(x.values))

In [4]:
#estrapolo l'anno dalla colonna release_date
df_movies['year'] = pd.to_datetime(df_movies['release_date'],format='%Y-%m-%d').dt.year

In [5]:
# punto 1 cast
cast = estrazione_json(credits, 'cast') 
cast_1 = cast.groupby('movie_id').size().to_frame().reset_index().rename(columns={0: 'crew'})
numero_cast = pd.merge(cast_1, credits[['title', 'movie_id']], on = 'movie_id')
numero_cast.head(6)

,movie_id,crew,title
0,5,24,Four Rooms
1,11,106,Star Wars
2,12,24,Finding Nemo
3,13,66,Forrest Gump
4,14,40,American Beauty
5,16,29,Dancer in the Dark


In [6]:
# punto 2
# How many movies do not have a homepage?
# seleziono tutti i film che non hanno una homepage
movies_without_homepage = df_movies[df_movies['homepage'].isnull()]
# conto il numero di film che ho selezionato in precedenza
len(movies_without_homepage)

3091

In [7]:
#punto 3
#faccio una groupby per anno sul dataset movies_without_homepage e conto con size 
movies_without_homepage_size = movies_without_homepage.groupby('year').size().to_frame().reset_index().rename(columns = {0 : 'dimensioni'}).head(6)
movies_without_homepage_size['year'] = movies_without_homepage_size['year'].astype(int)
movies_without_homepage_size.head(6)

,year,dimensioni
0,1916,1
1,1925,1
2,1927,1
3,1929,2
4,1930,1
5,1932,1


In [8]:
#punto 4 
#estraggo il dominio di tutte le homepage disponibili
from urllib.parse import urlparse
df = movies[movies['homepage'].notnull()].reset_index().drop('index', axis = 1)
domain = []
for i in range(len(df)):
    df_parse = urlparse(df.homepage[i])
    domain.append(df_parse)
domain_1 = pd.DataFrame(domain).netloc.str.split('.')
result = []
for i in range(len(df)):
    domain_1_parse = domain_1[i][-1]
    result.append(domain_1_parse)
result_1 = pd.DataFrame(result)
result_1 = pd.concat([df[['original_title', 'homepage']], result_1], axis=1).rename(columns = {0 : 'domain'})
result_1.head(6)


,original_title,homepage,domain
0,Avatar,http://www.avatarmovie.com/,com
1,Pirates of the Caribbean: At World's End,http://disney.go.com/disneypictures/pirates/,com
2,Spectre,http://www.sonypictures.com/movies/spectre/,com
3,The Dark Knight Rises,http://www.thedarkknightrises.com/,com
4,John Carter,http://movies.disney.com/john-carter,com
5,Spider-Man 3,http://www.sonypictures.com/movies/spider-man3/,com


In [9]:
# punto 5
movies_keyword = estrazione_json(df_movies, 'keywords')
keywords = movies_keyword[["id", "name"]].drop_duplicates()
keywords.id = keywords.id.astype(int)
keywords_1 = keywords.set_index("id")
movies_keyword = movies_keyword[["id","movie_id"]].reset_index().drop('index', axis = 1)
movies_keyword.columns = ["keyword_id", "movie_id"]
movies_keyword.keyword_id = movies_keyword.keyword_id.astype(int)
movies_keyword.head(6)

,keyword_id,movie_id
0,1463,19995
1,2964,19995
2,3386,19995
3,3388,19995
4,3679,19995
5,3801,19995


In [10]:
#punto 5
movies_genere = estrazione_json(df_movies, 'genres')
generi = movies_genere[["id", "name"]].drop_duplicates()
generi.id = generi.id.astype(int)
generi_1 = generi.set_index("id")
movies_genere = movies_genere[["id","movie_id"]].reset_index().drop('index', axis = 1).rename(columns = {'id':'genere_id'})
movies_genere.genere_id = movies_genere.genere_id.astype(int)
movies_genere.head(6)

,genere_id,movie_id
0,28,19995
1,12,19995
2,14,19995
3,878,19995
4,12,285
5,14,285


In [11]:
#punto 5
movies_aziende = estrazione_json(df_movies, 'production_companies')
aziende = movies_aziende[['id', 'name']].drop_duplicates()
aziende.id = aziende.id.astype(int)
aziende_1 = aziende.set_index('id')
movies_aziende = movies_aziende[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id':'aziende_id'})
movies_aziende.aziende_id = movies_aziende.aziende_id.astype(int)
movies_aziende.head(6)

,aziende_id,movie_id
0,289,19995
1,306,19995
2,444,19995
3,574,19995
4,2,285
5,130,285


In [12]:
#punto 5
production_nazioni = estrazione_json(df_movies, 'production_countries').rename(columns = {'iso_3166_1' : 'id'})
nazioni = production_nazioni[['id', 'name']].drop_duplicates()
nazioni_1 = nazioni.set_index('id')
production_nazioni = production_nazioni[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id':'nazioni_id'})
production_nazioni.head(6)

,nazioni_id,movie_id
0,US,19995
1,GB,19995
2,US,285
3,GB,206647
4,US,206647
5,US,49026


In [13]:
#punto 5 (lingue parlate)
lingue_parlate_movies = estrazione_json(df_movies, 'spoken_languages').rename(columns = {'iso_639_1' : 'id'})
lingue_parlate = lingue_parlate_movies[['id', 'name']].drop_duplicates()
lingue_parlate_1 = lingue_parlate.set_index('id')
lingue_parlate_movies = lingue_parlate_movies[['id', 'movie_id']].reset_index().drop('index', axis = 1).rename(columns = {'id': 'lingua_id'})
lingue_parlate_1.head(6)

,name
id,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch
tr,Türkçe


In [63]:
data = {'gender': [0.00,1.00,2.00],
        'Gender': ['Nan','F','M']}
df = pd.DataFrame(data)

In [58]:
#punto 5
cast_normalizzato = estrazione_json(credits, 'cast')
cast_normalizzato_1 = pd.merge(cast_normalizzato, df, on = 'gender').drop(['gender'], axis = 1)
cast_normalizzato_1.id = cast_normalizzato_1.id.astype(int)
cast_normalizzato_1.cast_id = cast_normalizzato_1.cast_id.astype(int)
cast_normalizzato_1.order = cast_normalizzato_1.order.astype(int)
attori = cast_normalizzato_1[['id', 'name', 'Gender']]
cast_credito = cast_normalizzato_1[['cast_id', 'character', 'credit_id', 'id', 'order']].set_index('credit_id')

In [59]:
#punto 5
crew_normalizzato = estrazione_json(credits, 'crew')
crew_normalizzato_1 = pd.merge(crew_normalizzato, df, on = 'gender').drop(['gender'], axis = 1)
crew_normalizzato_1.id = crew_normalizzato_1.id.astype(int)
lavoratori = crew_normalizzato_1[['id', 'name', 'Gender']]
crew_credito = crew_normalizzato_1[['id', 'department', 'credit_id', 'job', 'movie_id']].set_index('credit_id')

In [61]:
#punto 5 (persone unite)
personale = pd.concat([attori, lavoratori], axis = 0).drop_duplicates().set_index('id')


In [18]:
#punto 6
#creo una nuova colonna come differenza fra revenue e budget
df_movies['gross_margin'] = df_movies.revenue - movies.budget 
df_movies[['title','revenue','budget','gross_margin']].head(3)

,title,revenue,budget,gross_margin
0,Avatar,2787965087,237000000,2550965087
1,Pirates of the Caribbean: At World's End,961000000,300000000,661000000
2,Spectre,880674609,245000000,635674609


In [28]:
# punto 7
crew = estrazione_json(credits, 'crew')
crew_1 = crew.groupby('movie_id').size()
crew_2 = pd.DataFrame(crew_1).reset_index().rename(columns={0: 'lavoratori'})
numero_lavoratori = pd.merge(crew_2, credits[['title', 'movie_id']], on = 'movie_id')
numero_lavoratori.head(6)

,movie_id,lavoratori,title
0,5,88,Four Rooms
1,11,20,Star Wars
2,12,104,Finding Nemo
3,13,93,Forrest Gump
4,14,109,American Beauty
5,16,62,Dancer in the Dark


In [20]:
# numero di direttori
# punto 8
direttori =  estrazione_json(credits, 'crew')
direttori_1 = direttori[direttori['job'] == 'Director'].groupby('movie_id').size().to_frame().reset_index().rename(columns={0: 'Direttori'})
numero_direttori = pd.merge(direttori_1, credits[['title', 'movie_id']], on = 'movie_id')
numero_direttori.head(6)


,movie_id,Direttori,title
0,5,4,Four Rooms
1,11,1,Star Wars
2,12,1,Finding Nemo
3,13,1,Forrest Gump
4,14,1,American Beauty
5,16,1,Dancer in the Dark


In [21]:
# punto 9
# estrazione lingue
lingua = estrazione_json(df_movies, 'spoken_languages')
lingua_1 = lingua.groupby( 'iso_639_1').size().to_frame().reset_index().rename(columns={0: 'numero lingue'})
lingua_1.head(6)

,iso_639_1,numero lingue
0,af,7
1,am,1
2,ar,67
3,bg,3
4,bm,1
5,bn,1


In [40]:
# punto 10
movies_1 = df_movies[df_movies['year'].notnull()]
movies_1['decade'] = (movies_1['year'] / 10).astype(int)
company = estrazione_json(df_movies, 'production_companies')
company_1 = pd.merge(company, movies_1[['decade', 'title', 'movie_id', 'revenue', 'year']]).drop('id', axis = 1)
company_2 = company_1.groupby(['decade', 'name'])['revenue'].sum().to_frame().reset_index()
company_2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,decade,name,revenue
0,191,Triangle Film Corporation,8394751
1,191,Wark Producing Corp.,8394751
2,192,Metro-Goldwyn-Mayer (MGM),26358000
3,192,Nero Films,0
4,192,Paramount Pictures,650422


In [23]:
# punto 11
best_company = company_2.groupby(['decade'])['revenue'].transform(max) == company_2['revenue']
company_2[best_company]

# There are two major differences between the transform and apply groupby methods.
#apply implicitly passes all the columns for each group as a DataFrame to the custom function, while transform passes each column for each group as a Series to the custom function
#The custom function passed to apply can return a scalar, or a Series or DataFrame (or numpy array or even list). The custom function passed to transform must return a sequence (a one dimensional Series, array or list) the same length as the group.
#So, transform works on just one Series at a time and apply works on the entire DataFrame at once.

,decade,name,revenue
0,191,Triangle Film Corporation,8394751
1,191,Wark Producing Corp.,8394751
2,192,Metro-Goldwyn-Mayer (MGM),26358000
10,193,Metro-Goldwyn-Mayer (MGM),400176459
13,193,Selznick International Pictures,400176459
37,194,Walt Disney Productions,351747150
50,195,Metro-Goldwyn-Mayer (MGM),73770324
82,196,Eon Productions,598134765
248,197,United Artists,1392271577
450,198,Paramount Pictures,4431144486


In [24]:
#punto 12
#revenue minore
movies_revenue = df_movies[df_movies["gross_margin"] < 0].groupby(by = ("year")).size().to_frame().reset_index().rename(columns = {0 : 'numero film'})
movies_revenue['year'] = movies_revenue['year'].astype(int)
movies_revenue.head(6)

,year,numero film
0,1927,1
1,1947,1
2,1948,1
3,1954,1
4,1962,1
5,1963,1


In [25]:
# punto 13 
protagonisti = estrazione_json(credits, 'cast')
protagonisti_1 = pd.merge(protagonisti[['movie_id', 'name', 'order']], movies_1[['revenue', 'movie_id']], on = 'movie_id')
protagonisti_1['retribuzione'] = (protagonisti_1['revenue']/2)/(2**protagonisti_1['order'])
pd.set_option('display.float_format', lambda x: '%.2f' % x)
protagonisti_1['order'] = protagonisti_1['order'].astype(int)
protagonisti_1.head(6)

,movie_id,name,order,revenue,retribuzione
0,19995,Sam Worthington,0,2787965087,1393982543.50
1,19995,Zoe Saldana,1,2787965087,696991271.75
2,19995,Sigourney Weaver,2,2787965087,348495635.88
3,19995,Stephen Lang,3,2787965087,174247817.94
4,19995,Michelle Rodriguez,4,2787965087,87123908.97
5,19995,Giovanni Ribisi,5,2787965087,43561954.48


In [26]:
#punto 14
totale_retribuzione_attori = protagonisti_1.groupby('name').sum().reset_index().sort_values(by = 'retribuzione', ascending = False).drop(['revenue', 'order'], axis = 1)
totale_retribuzione_attori.head(11)

,name,movie_id,retribuzione
50772,Tom Cruise,775502,3976017174.37
50821,Tom Hanks,893665,3799897896.06
43611,Robert Downey Jr.,1031283,3631679406.09
25844,Johnny Depp,2056384,3511502683.80
53132,Will Smith,1473901,3281555344.09
4872,Ben Stiller,1401034,2832962893.16
11266,Daniel Radcliffe,863382,2779387774.39
6817,Bruce Willis,1425539,2575769647.62
31040,Leonardo DiCaprio,719968,2557793163.50
19411,Harrison Ford,938014,2509043297.27


In [27]:
#punto 15
totale_retribuzione_attori.loc[totale_retribuzione_attori['retribuzione'].idxmax()]


name              Tom Cruise
movie_id              775502
retribuzione   3976017174.37
Name: 50772, dtype: object